<a href="https://colab.research.google.com/github/tasjapr/diploma/blob/master/my_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preset

In [0]:
!rm -r /content/sample_data/

In [0]:
#check GPU

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
import os
os.chdir('/content/')

!git clone https://github.com/tensorflow/models.git

In [0]:
#connect to google drive

from google.colab import drive
drive.mount('/content/drive')

In [0]:
# copy necessary files from google drive

# !rm -r /content/files/*
!cp -r /content/drive/My\ Drive/Colab\ Notebooks/files/ /content/

In [0]:
os.chdir('/content/models/')
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
    
os.chdir('research/')

!protoc object_detection/protos/*.proto --python_out=.

# Crop and align 

In [0]:
!apt install findutils
!apt install coreutils

In [0]:
os.chdir('/content/')
!git clone https://github.com/cmusatyalab/openface.git

In [0]:
os.chdir('/content/openface/')

!python2 setup.py install

In [0]:
os.chdir('/content/openface/models/')

!./get-models.sh
!mkdir ../training-images

In [0]:

!cp -r /content/files/images/* /content/openface/training-images/
!cp -r /content/files/test_images/ /content/openface/training-images/

In [0]:
import os
os.chdir('/content/openface/')

# !./util/align-dlib.py ./training-images/ align outerEyesAndNose ./aligned-images/ --size 96
!./util/align-dlib.py ./training-images/ align outerEyesAndNose ./aligned-images/ --size 96

#TensorFlow object detection API install

https://colab.research.google.com/drive/1dU0bKSzYt8mz83lsI09tMhn-A7w1Y9MY#scrollTo=CWuJDoKJHMSs&forceEdit=true&offline=true&sandboxMode=true

In [0]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

In [0]:
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI; make; cp -r pycocotools /content/models/research/

In [0]:
import os
os.chdir('/content/models/research')

%set_env PYTHONPATH=/content/models/research:/content/models/research/slim
!protoc object_detection/protos/*.proto --python_out=.
# !pip install 'prompt-toolkit==1.0.15'

In [0]:
# check installation

!python object_detection/builders/model_builder_test.py

# generate tf_record and csv file

In [0]:
# if you want to use aligned images for training

# !rm -r /content/files/images/*
# !mv /content/openface/aligned-images/* /content/files/images/

In [0]:
#create xml file for each image

# os.chdir('/content/files/')
# !python3 create_xml.py

In [0]:
os.chdir('/content/files/')
!python3 xml_to_csv.py

In [0]:
!python3 generate_tfrecord.py --csv_input=/content/files/data/train_labels.csv --output_path=/content/files/data/train.record --image_dir=/content/files/images/train/
!python3 generate_tfrecord.py --csv_input=/content/files/data/test_labels.csv --output_path=/content/files/data/test.record --image_dir=/content/files/images/test/

# Get last checkpoint

In [0]:
!mkdir /content/models/research/object_detection/training/

os.chdir('/content/files/')
import zipfile, os
print ("Uncompressing zip file")
zip_ref = zipfile.ZipFile('training.zip', 'r')
zip_ref.extractall('/content/models/research/object_detection/training/')
zip_ref.close()

In [0]:
!mv /content/models/research/object_detection/training/content/models/research/object_detection/training/* /content/models/research/object_detection/training/
!rm -r /content/models/research/object_detection/training/content/

# Training model

In [0]:
#for change .config file

# !mv /content/ssd_mobilenet_v2_coco.config /content/files/
# !cp /content/files/ssd_mobilenet_v2_coco.config /content/models/research/object_detection/training/

In [0]:
os.chdir('/content/models/research/object_detection/')

!mkdir training

In [0]:
!cp -r /content/files/data/ /content/models/research/object_detection/
!cp -r /content/files/images/ /content/models/research/object_detection/
!cp -r /content/files/ssd_mobilenet_v2_coco.config /content/models/research/object_detection/training/

In [0]:
import os
os.chdir('/content/models/research/object_detection/legacy/')

!python3 train.py --logtostderr \
                  --train_dir=../training/ --pipeline_config_path=../training/ssd_mobilenet_v2_coco.config

In [0]:
# create zip file with all of training`s data

!zip -r /content/training.zip /content/models/research/object_detection/training/

In [0]:
# create zip file with all of end data

!zip -r /content/emotion_graph.zip /content/models/research/object_detection/emotions_graph/

# Model evaluation

 **Before this block** you must to change some params in 

> *object_detection_evaluation.py: *



Replace all `unicode` function with String `str` function



---

*ssdlite_mobilenet_v2_coco.config:*

```
eval_config: {
        #num of test images.Previously It was 8000? change to your number of test images
        num_examples: 7
        # Note: The below line limits the evaluation process to 10 evaluations.
        # Remove the below line to evaluate indefinitely.
        max_evals: 10
      }
```



In [0]:
!cp /content/drive/My\ Drive/Colab\ Notebooks/files/object_detection_evaluation.py /content/models/research/object_detection/utils/

In [0]:
!cp /content/drive/My\ Drive/Colab\ Notebooks/files/ssdlite_mobilenet_v2_coco.config /content/models/research/object_detection/training/

In [0]:
import  os
os.chdir('/content/models/research/object_detection/legacy/')

!python3 eval.py \
    --logtostderr \
    --pipeline_config_path=../training/ssdlite_mobilenet_v2_coco.config \
    --checkpoint_dir=../training/ \
    --eval_dir=eval/

In [0]:
# !rm -r /content/models/research/object_detection/legacy/eval/

In [0]:
# !mv /content/anger.png /content/files/images/test/
# !mv /content/anger.xml /content/files/images/test/
# !mv /content/dis.png /content/files/images/test/
# !mv /content/dis.xml /content/files/images/test/
# !mv /content/surprise.png /content/files/images/test/
# !mv /content/surprise.xml /content/files/images/test/

# Visualizing results

In [0]:
!mkdir /content/models/research/object_detection/test_images/

# !cp -r /content/openface/aligned-images/* /content/models/research/object_detection/test_images/
!cp -r /content/files/test_images_cr/* /content/models/research/object_detection/test_images/

In [0]:
#If you have final ckpt - upload ckpt files and press 'run cell'

# !mv /content/model.ckpt-7000.index /content/models/research/object_detection/training/
# !mv /content/model.ckpt-7000.meta /content/models/research/object_detection/training/

In [0]:
!mv /content/models/research/object_detection/test_images/image63.jpg /content/models/research/object_detection/test_images/image34.jpg
!mv /content/models/research/object_detection/test_images/image65.jpg /content/models/research/object_detection/test_images/image37.jpg
!mv /content/models/research/object_detection/test_images/image64.jpg /content/models/research/object_detection/test_images/image38.jpg


In [0]:
os.chdir('/content/models/research/object_detection/')
!python3 export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path training/ssd_mobilenet_v2_coco.config \
    --trained_checkpoint_prefix training/model.ckpt-50000 \
    --output_directory emotions_graph

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
import matplotlib.pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')

In [0]:
# This is needed to display the images.
%matplotlib inline

In [0]:
from utils import label_map_util
from utils import visualization_utils as vis_util

In [0]:
# What model to download.
MODEL_NAME = 'emotions_graph'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('/content/models/research/object_detection/data/', 'object-detection.pbtxt')

NUM_CLASSES = 7

In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [0]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [0]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.png'.format(i)) for i in range(3, 65) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (10, 6)

In [0]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
 # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [0]:
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path).convert('RGB')
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=False,
      line_thickness=1)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)

# Change model

In [0]:
# !rm /content/models/research/object_detection/data/ssdlite_mobilenet_v2_coco.config
# !rm /content/models/research/object_detection/training/ssdlite_mobilenet_v2_coco.config

!mv /content/ssd_mobilenet_v2_coco.config /content/files/data/
# now go to 'training model'


# TensorBoard

In [0]:
!pip install tensorboardcolab

from tensorboardcolab import *
tbc=TensorBoardColab()

In [0]:
!tensorboard --logdir='/content/models/research/object_detection/training'

In [0]:
# !pip install -U tensorboardcolab
# from tensorboardcolab import *
# import shutil

# #clean out the directory
# shutil.rmtree('./Graph', ignore_errors=True)
# os.mkdir('./Graph')

# tf.reset_default_graph()

# #will start the tunneling and will print out a link:
# tbc=TensorBoardColab()

# #**here you construct your model**

# sess = tf.Session()
# output = sess.run(....)
# sess.close()

# train_writer = tbc.get_writer();
# train_writer.add_graph(sess.graph)

# train_writer.flush();
# tbc.close()

In [0]:
# !pip install -q tf-nightly-2.0-preview
# # Load the TensorBoard notebook extension
# %load_ext tensorboard

In [0]:
# import tensorflow as tf
# import datetime, os